# Si-Ge cluster expansion workflow - part 2

This is a CASM project tutorial to generate a phase diagram using a Si-Ge binary alloy cluster expansion fit to DFT calculations. The overall workflow is split into two parts.

Topics covered in part 2:

1. **Basis set construction**: Specify clusters and basis functions and construct a Clexulator
2. **Cluster expansion fitting**: Collect energies from import and mapping results and evaluate correlations, the per unitcell mean value of the symmetrically equivalent cluster functions. Fit coefficients to DFT calculated energies
4. **Monte Carlo simulations**: Run semi-grand canonical Monte Carlo simulations using the cluster expansion

** **Important** **: The following step uses *casm.bset.autoconfigure* to set CASM environment variables automatically if CASM_PREFIX is not already set. The instructions [here](https://prisms-center.github.io/CASMcode_pydocs/casm/bset/2.0/installation.html#environment-variable-configuration) describe how to set CASM environment variables before launching Jupyter.


## Setup

### Imports and paths

In [ ]:
import pathlib
import libcasm.xtal as xtal
from libcasm.xtal import pretty_json
from casm.project import Project
from casm.project.json_io import read_required, safe_dump

input_dir = pathlib.Path("input")
project_path = pathlib.Path("SiGe_occ")

# Configure environment variables:
import os

if "CASM_PREFIX" not in os.environ:
    import casm.bset

    print("Autoconfigure...")
    casm.bset.autoconfigure()
    print("Autoconfigure DONE")

### Setup checks

- This notebook depends the import results obtained in part 1. Here we check that the necessary results exist. 

In [ ]:
# Construct project & read import results from part 1
try:
    project = Project.init(path=project_path)
    data_dir = project.path / "precalculated"
    mapped_structures_path = data_dir / "mapped_structures.json"
    mapped_structures = read_required(mapped_structures_path)
    if len(mapped_structures) != 119:
        raise ValueError(
            f"Expected 119 mapped structures, found {len(mapped_structures)}. "
            "Try removing the SiGe_occ directory and re-running part 1."
        )
except Exception as e:
    print(e)
    raise ValueError("Make sure to run part 1 first.")

print(f"Found {len(mapped_structures)} mapped structures")

## Basis set construction

### Basis set generating group

The formation energy is invariant under transformation by prim factor group operations, so we call it the "generating group" for our cluster expansion basis set. 

- [Project.sym]() gives quick access to symmetry information for the project.
- [Project.sym.print_factor_group]() gives a summary of the prim factor group operations.


In [ ]:
project.sym.print_factor_group()

### Specify clusters and basis functions

Use [*bset.make_bspecs*]() to construct a cluster expansion basis set for the Si-Ge formation energy. The parameters that may be useful are:

- *max_length*: The maximum site-to-site distance to allow in clusters, by number of sites in the cluster.
    - Example: ``max_length=[0.0, 0.0, 5.0, 4.0]`` specifies that pair clusters up to distance 5.0 and triplet clusters up to distance 4.0 should be included. The null cluster and point cluster values (elements 0 and 1) are arbitrary.
- *custom_generators*: Optionally, specify particular clusters to include regardless of *max_length*
- *occ_site_basis_functions_specs*: Select the occupation site basis functions. The most common options are:
  - "chebychev": An expansion (with correlation values all equal to 0) about the idealized random alloy where the probability of any of the allowed occupants on a particular site is the same.
  - "occupation": An expansion (with correlation values all equal to 0) about the default configuration where each site is occupied by the first allowed occupant in the Prim.occ_dof list.
  - See details [here](https://prisms-center.github.io/CASMcode_pydocs/casm/bset/2.0/usage/basis_function_specs.html#occupation-site-basis-functions)

In [ ]:
# Specify the basis set ID
# - Must be alphanumeric and underscores only
bset_id = "default"

# Specify maximum cluster site-to-site distance,
# by number of sites in the cluster
pair_max_length = 10.01
triplet_max_length = 7.27
quad_max_length = 4.0

# Use chebychev site basis functions (+x, -x)
occ_site_basis_functions_specs = "chebychev"

project.bset.make_bspecs(
    id=bset_id,
    max_length=[
        0.0, # null cluster, arbitrary
        0.0, # point cluster, arbitrary
        pair_max_length,
        triplet_max_length,
        quad_max_length,
    ],
    occ_site_basis_functions_specs="chebychev",
    force=True,
)

### The bspecs.json file

The previous steps created a "bspecs.json" file storing the basis set specifications:

- *cluster_specs*: specifications for which clusters to construct 
  cluster functions on 
- *basis_functions_specs*: specifications for which type of basis
  functions to generate
- *version*: specifies the Clexulator version to write (CASM v2+)

The "bspecs.json" file can also be edited manually, using the format described [here](https://prisms-center.github.io/CASMcode_docs/formats/casm/clex/ClexBasisSpecs/).


In [ ]:
bspecs_path = project.dir.bspecs(bset="default")
print(pretty_json(read_required(bspecs_path)))

### Generate and compile the Clexulator

The *bset.update* command generates and compiles a Clexulator (cluster expansion calculator).

Options include:

- *no_compile*: generate the basis set so that you can inspect the clusters and functions without compiling the Clexulator
- *only_compile*: re-compile a Clexulator using the existing files

In [ ]:
project.bset.update(
    id=bset_id,
    # no_compile=False,
    # only_compile=False
)

### Inspect the cluster expansion

Print the cluster orbits:

- An "orbit" is the set of symmetrically equivalent objects. The "prototype" is one element in the orbit.
- In the context of periodic cluster expansion, the "multiplicity" of the orbit is the number of equivalent per unit cell (avoiding double counting clusters which include sites in multiple unit cell).
- The "cluster invariant group" is the set of prim factor group operations plus some lattice translation which leave the cluster unchanged.
  - This is the symmetry used to construct cluster functions. 

In [ ]:
bset_id="default"
project.bset.print_orbits(
    id=bset_id,
    linear_orbit_indices=None,  # use i.e. set(range(1,5)) to print a subset
)

Print the cluster function prototypes:

- These are the cluster functions on the prototype cluster
- For a binary alloy like Si-Ge there is one function per cluster
  - For example, if the default occupation is Si, then the cluster expansion includes terms for Ge (point), Ge-Ge (pair), Ge-Ge-Ge (triplet), etc., but no Si-Ge (pair) term, because that is the same a Ge (point) term
- In general there may be >1 cluster per function, to account for interactions between different combinations of occupations 

In [ ]:
project.bset.print_functions(
    id=bset_id,
    linear_orbit_indices=None,  # use i.e. set(range(1,5)) to print a subset
)

# \phi_{sublattice_index, function_index}
# value[function_index][occupant_index]

## Monte Carlo simulations

### Load a Monte Carlo System

In [ ]:
from casm.project.json_io import read_required
from libcasm.clexmonte import (
    System,
)

system_data = read_required(input_dir / "system.json")
bset_dir = project.dir.bset_dir(bset=bset_id)

system = System.from_dict(
    data=system_data,
    search_path=[str(input_dir), str(bset_dir)],
)

### Run simulations

In [ ]:
from libcasm.clexmonte import MonteCalculator, make_initial_state

output_dir = project.path / "output"
output_dir.mkdir(parents=True, exist_ok=True)
summary_file = output_dir / "summary.json"

# construct a semi-grand canonical MonteCalculator
calculator = MonteCalculator(
    method="semigrand_canonical",
    system=system,
)

# construct default sampling fixture parameters
thermo = calculator.make_default_sampling_fixture_params(
    label="thermo",
    output_dir=str(output_dir),
)
print(xtal.pretty_json(thermo.to_dict()))

# construct the initial state (default configuration)
initial_state, motif, motif_id = make_initial_state(
    calculator=calculator,
    conditions={
        "temperature": 300.0,
        "param_chem_pot": [-1.0],
    },
    min_volume=1000,
)

# Run
sampling_fixture = calculator.run_fixture(
    state=initial_state,
    sampling_fixture_params=thermo,
)

In [ ]:
import numpy as np

# construct the initial state (default configuration)
initial_state, motif, motif_id = make_initial_state(
    calculator=calculator,
    conditions={
        "temperature": 300.0,
        "param_chem_pot": [-1.0],
    },
    min_volume=1000,
)

state = initial_state

# Run several, w/ dependent runs
mu_list = np.arange(-4.0, 0.01, step=0.5)
for mu in mu_list:
    state.conditions.vector_values["param_chem_pot"] = [mu]
    sampling_fixture = calculator.run_fixture(
        state=state,
        sampling_fixture_params=thermo,
    )